## Trabalho prático 2 - Exercício 2

Para este exercício foi proposta a implementação de um algoritmo 
de assinatura **EdCDSA**.
Este designa-se por *Edwards-curve Co-factor Digital Signature Algorithm*, 
sendo um esquema de assinatura baseado em **curvas elíipticas**, mais concretamente *Twisted Edwards-curve*,
baseado no algoritmo **ECDSA**(*Elliptic Curve Digital Signature Algorithm*);
bastante semelhante ao **EdDSA**.

Em três pontos distintos do enunciado, é pedido que:
- A implementação tenha funções para assinar digitalmente e verificar a assinatura.
- A implementação dever usar uma das *Twisted Edwards Curves* definidas no *standard*, escolhidas na iniciação da classe: **Ed25519** e **Ed448**.
- Utilizar a transformação de *Fiat-Shamir* para construir um protocolo de autenticação de desafio-reposta.

Deste modo, para implementar o **EdCDSA**, 
é necessário utilizar uma das curvas de *Twisted Edwards* definidas no **FIPS186-5**, 
que pode ser a curva **edwards25519** ou **edwards448**; deixando a escolha ao utilizador.

Além disso, deve-se aplicar a transformação de **Fiat-Shamir** para a autenticação.
Este é um método para transformar um esquema de assinatura de conhecimento zero interativo (*SNARK*) em uma versão não interativa; altamente utilizado na criptografia moderna.

O protocolo é baseado no uso de funções *hash* criptográficas. 
O processo implica o provador enviar uma série de mensagens ao verificador para provar a validade de uma determinada afirmação. 
No protocolo de *Fiat-Shamir*, essas mensagens são substituídas por valores *hash* das mesmas.
O verificador então usa os valores *hash* como se fossem as mensagens originais e continua a verificar a afirmação.

In [1]:
# imports
import hashlib, os
import random 
from pickle import dumps
from struct import *

## Funções auxiliares fornecidas pela equipa docente

## Classe de implementação da curva de Edwards

O *EdCDSA* utiliza curvas de *Edwards*, que são uma classe 
especial de curvas elípticas que têm a forma de uma equação 
de *Edwards*:

```bash
x^2 + y^2 = 1 + dx^2y^2
```

... onde **d** é um parâmetro constante. 
As curvas de Edwards são conhecidas por serem mais eficientes do que as 
curvas elípticas tradicionais para certas operações, como a adição de 
pontos e a verificação de pontos na curva.

O algoritmo *EdCDSA* utiliza as curvas de *Edwards* 
para gerar chaves e assinaturas da mesma forma que 
o algoritmo *DSA* utiliza as curvas elípticas.

## Classe *Ed*

A classe seguinte é fornecida pela equipa docente.

A classe *Ed* é usada para definir uma curva elíptica 
de *Edwards* sobre um **corpo finito**, e contém métodos 
para mapear pontos nesta curva para uma **curva elíptica**.

O construtor da classe *Ed* recebe quatro parâmetros:
- um primo p;
- os coeficientes a e d da curva de Edwards;
- o parâmetro opcional *ed*.

Se o parâmetro opcional *ed* é fornecido, ele deve ser um 
dicionário com as chaves *'L', 'Px' e 'Py'*, contendo respetivamente 
o **tamanho do maior subgrupo da curva**, e as **coordenadas x e y de um 
ponto na curva de *Edwards*** que será usado como **gerador do grupo**. 
Caso contrário, a função *gen()* é usada para gerar um ponto aleatório
na curva.

No método *init()*, são calculados os coeficientes da 
curva de *Edwards* a partir dos coeficientes **a e d**;
define-se também a curva elíptica *EC*, com os 
coeficientes **a4 e a6**.
O método *order()* calcula a **ordem n do maior subgrupo da 
curva EC** e o **cofator h**; e retorna-os num tuplao *(n,h)*.
O método *gen()* usa o método *order()* para gerar um ponto 
aleatório **P** na curva *EC* que não está no subgrupo trivial, 
e define **self.P** como o ponto **h*P** e **self.L** como o tamanho 
do maior subgrupo.
O método *is_edwards(x,y)* verifica se um ponto *(x,y)* 
está na curva de *Edwards* definida pelos **coeficientes a e d**.
O método *ed2ec(x,y)* mapeia um ponto **(x,y)** na curva de *Edwards* 
para um ponto na curva elíptica de *Weierstrass* **EC**.
O método *ec2ed(P)* mapeia um ponto **P na curva EC** para um ponto na 
curva de *Edwards*. 

In [2]:
class Ed(object):
    def __init__(self,p, a, d , ed = None):
        assert a != d and is_prime(p) and p > 3
        K        = GF(p) 
  
        A =  2*(a + d)/(a - d)
        B =  4/(a - d)
    
        alfa = A/(3*B) ; s = B

        a4 =  s^(-2) - 3*alfa^2
        a6 =  -alfa^3 - a4*alfa
        
        self.K = K
        self.constants = {'a': a , 'd': d , 'A':A , 'B':B , 'alfa':alfa , 's':s , 'a4':a4 , 'a6':a6 }
        self.EC = EllipticCurve(K,[a4,a6]) 
        
        if ed != None:
            self.L = ed['L']
            self.P = self.ed2ec(ed['Px'],ed['Py'])  # gerador do grupo
        else:
            self.gen()
    
    def order(self):
        # A ordem prima "n" do maior subgrupo da curva, e o respetivo cofator "h" 
        oo = self.EC.order()
        n,_ = list(factor(oo))[-1]
        return (n,oo//n)
    
    def gen(self):
        L, h = self.order()       
        P = O = self.EC(0)
        while L*P == O:
            P = self.EC.random_element()
        self.P = h*P ; self.L = L

    def is_edwards(self, x, y):
        a = self.constants['a'] ; d = self.constants['d']
        x2 = x^2 ; y2 = y^2
        return a*x2 + y2 == 1 + d*x2*y2

    def ed2ec(self,x,y):      ## mapeia Ed --> EC
        if (x,y) == (0,1):
            return self.EC(0)
        z = (1+y)/(1-y) ; w = z/x
        alfa = self.constants['alfa']; s = self.constants['s']
        return self.EC(z/s + alfa , w/s)
    
    def ec2ed(self,P):        ## mapeia EC --> Ed
        if P == self.EC(0):
            return (0,1)
        x,y = P.xy()
        alfa = self.constants['alfa']; s = self.constants['s']
        u = s*(x - alfa) ; v = s*y
        return (u/v , (u-1)/(u+1))


### Classe de implementação dos métodos dos pontos de edwards

Outra classe fornecida pela equipa docente.

A classe **ed** define um ponto na curva de *Edwards* e 
contém métodos para operações de soma, duplicação e 
multiplicação escalar, para as operações necessárias nas 
funções a serem implementadas pela classe **EdCDSA**.

In [3]:
class ed(object):
    def __init__(self,pt=None,curve=None,x=None,y=None):
        if pt != None:
            self.curve = pt.curve
            self.x = pt.x ; self.y = pt.y ; self.w = pt.w
        else:
            assert isinstance(curve,Ed) and curve.is_edwards(x,y)
            self.curve = curve
            self.x = x ; self.y = y ; self.w = x*y
    
    def eq(self,other):
        return self.x == other.x and self.y == other.y
    
    def copy(self):
        return ed(curve=self.curve, x=self.x, y=self.y)
    
    def zero(self):
        return ed(curve=self.curve,x=0,y=1)
    
    def sim(self):
        return ed(curve=self.curve, x= -self.x, y= self.y)
    
    def soma(self, other):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*self.w*other.w
        self.x, self.y  = (self.x*other.y + self.y*other.x)/(1+delta), (self.y*other.y - a*self.x*other.x)/(1-delta)
        self.w = self.x*self.y
        
    def duplica(self):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*(self.w)^2
        self.x, self.y = (2*self.w)/(1+delta) , (self.y^2 - a*self.x^2)/(1 - delta)
        self.w = self.x*self.y
        
    def mult(self, n):
        m = Mod(n,self.curve.L).lift().digits(2)   ## obter a representação binária do argumento "n"
        Q = self.copy() ; A = self.zero()
        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()
        return A

### Classe que implementa as assinaturas EdCDSA

Nesta classe são implementadas **3 funcionalidades** base 
dos algoritmos de assinatura: 
- a geração de chaves (generateKey);
- assinatura (signature);
- verificação da assinatura (verify).

#### Geração de chaves:
O primeiro passo é a geração de um par de chaves pública-privada. 

A chave privada é um número inteiro aleatório, geralmente escolhido 
com ajuda de um gerador de números pseudo-aleatórios criptográficos. 
A chave pública é um ponto na curva de *Edwards*, que é obtido através
da **multiplicação da chave privada pelo ponto gerador da curva**. 
O ponto gerador é um ponto pré-definido na curva, que é escolhido 
de tal forma que sua ordem seja um número primo grande, para garantir 
a segurança do algoritmo.

No entanto, as curvas de *Edwards* têm um **cofator** (um fator que divide 
a ordem da curva) diferente de 1. 
Isso significa que nem todos os pontos na curva são válidos como chaves 
públicas, e é necessário garantir tal.

Deste modo, após gerar uma chave privada **k** aleatória e calcular 
a chave pública **Q** como **Q = kG**. Em seguida, multiplique a chave pública 
pelo cofator **h** (que é um inteiro positivo que divide a ordem da curva), 
para obter a chave pública final **Q' = hQ**. Se **Q'** não for um ponto válido 
na curva de Edwards, o processo é repetido com uma nova chave privada **k** até 
obter uma chave pública válida.

#### Assinatura:
Para assinar uma mensagem, o remetente precisa seguir os seguintes passos:
- **Cálculo do hash da mensagem:** a mensagem é transformada em um valor hash criptográfico usando uma função hash segura, como SHA-256;
- **Escolha de um número aleatório k:** um número aleatório k é escolhido, novamente com a ajuda de um gerador de números pseudo-aleatórios criptográficos. O valor de k deve ser mantido em segredo e não deve ser reutilizado para outras assinaturas;
- **Cálculo do ponto de assinatura R:** o ponto de assinatura R é obtido multiplicando o valor de k pelo ponto gerador da curva. Esse ponto é um ponto na curva de *Edwards* e é utilizado para calcular a assinatura.
- **Cálculo do valor s:** o valor s é calculado usando a chave privada do remetente e o hash da mensagem. Ele é dado por s = (r + h*d)/k, onde r é a coordenada x do ponto R, d é a chave privada e h é o valor hash da mensagem.
- **Cálculo da assinatura:** a assinatura é um par de valores (R,s), que é enviado junto com a mensagem.

#### Verificação da assinatura:
Para verificar a assinatura, o destinatário precisa seguir os seguintes passos:
- **Cálculo do hash da mensagem:** a mensagem é transformada em um valor hash criptográfico usando a mesma função hash segura utilizada pelo remetente.
- **Verificação do ponto de assinatura R:** o destinatário verifica se o ponto de assinatura R pertence à curva de Edwards e se sua ordem é um número primo grande.
- **Cálculo do valor w:** o valor w é calculado usando o valor hash da mensagem e o valor s da assinatura. Ele é dado por w = s^(-1)*h.
- **Cálculo do ponto Q:** o ponto Q é obtido multiplicando o valor w pelo ponto gerador da curva e subtraindo o ponto


Em concreto, o algoritmo **EdCDSA** é uma implementação do 
**ECDSA** que usa curvas elípticas torcidas de *Edwards*. 


In [4]:
class EdCDSA():
    
    #Função de inicialização das variaveis a usar nos métodos
    def __init__(self, curve):
        self.E, self.G, self.b, self.l, self.p = self.setup(curve)
    
    #Parâmetros das curvas de ED25519 ou ED448
    def setup(self, curve):
        
        if curve == "ed25519":
            p = 2^255-19              # número primo que define o corpo sobre o qual a curva elíptica é definida
            K = GF(p)                 # Corpo finito de ordem p
            a = K(-1)                 # Coeficiente da equação da curva elíptica
            d = -K(121665)/K(121666)  # Coeficiente da equação da curva elíptica

            ed25519 = {
            'b'  : 256, # integer representing the bit-length of the prime field p
            'Px' : K(15112221349535400772501151409588531511454012693041857206046113283949847762202), # x-coordinate of a point on the edwards25519 curve
            'Py' : K(46316835694926478169428394003475163141307993866256225615783033603165251855960), # y-coordinate of a point on the edwards25519 curve
            'L'  : ZZ(2^252 + 27742317777372353535851937790883648493), # order of a subgroup of the edwards25519 curve
            'n'  : 254, # integer representing the bit-length of L
            'h'  : 2^3  # integer representing the cofactor of the edwards25519 curve
            }

            Bx = ed25519['Px']; By = ed25519['Py']

            E = Ed(p,a,d,ed=ed25519)  # an instance of the Ed class representing the edwards25519 curve
            b = ed25519['b']
            G = ed(curve=E,x=Bx,y=By) # a point on the edwards25519 curve used as a generator for the EdCDSA algorithm
            l = E.order()[0]          # the order of the edwards25519 curve
        
        else:
            p = 2^448 - 2^224 - 1
            K = GF(p)
            a = K(1)
            d = K(-39081)
            
            ed448= {
            'b'  : 456,     ## tamanho das assinaturas e das chaves públicas
            'Px' : K(224580040295924300187604334099896036246789641632564134246125461686950415467406032909029192869357953282578032075146446173674602635247710) ,
            'Py' : K(298819210078481492676017930443930673437544040154080242095928241372331506189835876003536878655418784733982303233503462500531545062832660) ,                                          
            'L'  : ZZ(2^446 - 13818066809895115352007386748515426880336692474882178609894547503885) ,
            'n'  : 447,     ## tamanho dos segredos: os dois primeiros bits são 0 e o último é 1.
            'h'  : 4        ## cofactor
            }
            
            Bx = ed448['Px']; By = ed448['Py']
        
            E = Ed(p,a,d,ed=ed448)
            b = ed448['b']
            G = ed(curve=E,x=Bx,y=By)
            l = E.order()[0]
        
        return E, G, b, l, p
    
    #Função de hash a ser utilizada nas curvas
    def hash512(self,data):
        
        return hashlib.sha512(data).digest()
        
    def hash256(self,data):
        
        return hashlib.shake_256(data).digest(114)
        
    #Função que determina a hash da chave privada 
    def digest(self,d):
        
        if self.b == 256:
            h = self.hash512(d)
        else:
            h = self.hash256(d)
            
        return h
    
    # Método para gerar o par de chaves pública/privada
    def generateKeys(self):
        
        # Co-fator da curva
        (n, h) = self.E.order()
            
        # Variável para verificar se o ponto pertence à curva
        is_ed = False
        
        while is_ed == False:
            
            # Gerar private key
            d = random.randint(1, n-1)
            
            # Gerar public key
            Q = self.G.mult(d)
    
            if self.E.is_edwards(Q.x, Q.y):
                is_ed = True
            
        return d, Q
            
    # Método para gerar a assinatura de uma mensagem
    def signature(self,m,d):
        
        # Ordem n da curva 
        (n, h) = self.E.order()
        
        # Calcular hash da mensagem
        h = self.digest(m)
        
        if log(n,2).n() >= 512:
            E = h
            e = int.from_bytes(E, 'little')
        else:
            E = h[:floor(log(n,2).n() / 8)]
            e = int.from_bytes(E, 'little')
        
        s = 0
        r = 0
        while s==0 or r==0:
            # Obter inteiro aleatório entre 1 e n-1
            k = random.randint(1, n-1)

            #Inversa modular de k
            inv_mod_k = inverse_mod(k, n)

            # Calcular ponto na curva (x,y) = k*G
            R = self.G.mult(k)

            # Cálculo de r
            r = int(R.x) % n
            
            # Cálculo de s
            s = (inv_mod_k * (e + r * d)) % n
            
            # Destruir K e a sua inversa modular
            k = None
            inv_mod_k = None

        return (r,s)
        
    # Método para verificar a mensagem da assinatura
    def verify(self,m,A,Q):
        
        (r,s) = A
        
        # Ordem n da curva 
        (n, _) = self.E.order()
        
        if 1 <= r <= n-1 and 1 <= s <= n-1:
            
            # Calcular hash da mensagem
            h = self.digest(m)    
            
            if log(n,2).n() >= 512:
                E = h
                e = int.from_bytes(E, 'little')
            else:
                E = h[:floor(log(n,2).n() / 8)]
                e = int.from_bytes(E, 'little')
            
            # Obter a inversa modular de s
            inv_s = inverse_mod(s, n)
            
            u = (e * inv_s) % n
            v = (r * inv_s) % n
            
            # Calcular o ponto da curva
            ug = self.G.mult(u)
            vq = Q.mult(v)
            
            vq.soma(ug)
            
            if vq.w == 0:
                raise ValueError("Signature Error")
            
            r1 = int(vq.x)
            rx = r1 % n
            
            if r == rx:
                return True
            else:
                return False
        else:
            raise ValueError("Signature Error")
            
    # Função que calcula a resposta para um dado desafio
    def calculate_response(self, challenge):
        # Gera uma resposta aleatória de 32 bytes
        response = os.urandom(32)
        # Calcula o hash da concatenação da mensagem original e a resposta
        hashed = hashlib.sha256(dumps(message1) + response).digest()
        return response, hashed

## Testes

In [5]:
edcdsa = EdCDSA("ed25519")
message1 = "Mensagem a ser assinada"
message2 = "Outra mensagem de teste"

print("Iniciar programa com mensagem" + message1)

prv_key, pub_key = edcdsa.generateKeys()
print("Private key: ")
print(prv_key)
print()
print("Public key: ")
print(pub_key)
print()

# Provar recebe o desafio do verificador
challenge = "Desafio aleatório"
print("Desafio recebido pelo provar:")
print(challenge)

# Provar calcula a resposta e a assinatura
response, hashed = edcdsa.calculate_response(challenge)
signature = edcdsa.signature(hashed, prv_key)

# Provar envia a resposta e a assinatura para o verificador
print("Resposta enviada para o verificador:")
print(response)
print("Assinatura digital enviada para o verificador:")
print(signature)

# Verificador verifica a autenticidade da mensagem
hashed = hashlib.sha256(dumps(message1) + response).digest()
if edcdsa.verify(hashed, signature, pub_key):
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada!")

assinatura = edcdsa.signature(dumps(message1), prv_key)
print("Assinatura: ")
print(assinatura)
print()

if edcdsa.verify(dumps(message1), assinatura, pub_key):
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada!")
    
print()
    
print("Verificação da autenticação da mensagem não enviada")
if edcdsa.verify(dumps(message2), assinatura, pub_key):
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada!")
    

Iniciar programa com mensagemMensagem a ser assinada
Private key: 
4362318791450537621865609424594444378914445545732071090006214368014761505332

Public key: 

Desafio recebido pelo provar:
Desafio aleatório
Resposta enviada para o verificador:
b"\xa3#\xadi\xe5\xdc\x17c:\x8e\x11L\xd1j\xfax&u\xf8F.n\x0f\xbc\x9a+\xd2'\xe6\xe8\x90="
Assinatura digital enviada para o verificador:
(1678059019948503655304789122327147642076626034884549415932691598973566691517, 2651738774295784559188337324022954635735051579158231350149242701044834854780)
Mensagem autenticada!
Assinatura: 
(1781600579250474621196941859811587781107139511033189565534570533118704694422, 3903120059825896354420553589791252085642650705332873936930016943227330626638)

Mensagem autenticada!

Verificação da autenticação da mensagem não enviada
Mensagem não autenticada!
